<a href="https://colab.research.google.com/github/iliya1984/DevML/blob/main/computer_vision/face_recognition/yale_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from PIL import Image
import cv2
import numpy as np

import zipfile
import os
from os import path
import json

from google.colab import drive
from google.colab.patches import cv2_imshow
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%run '/content/drive/MyDrive/Workspace/Computer Vision/cv_common.ipynb'

Common library was successfully loaded


## Load and extract directories definition


In [ ]:
train_zip_file_path = f'{datasets_directory}/yale_faces/train.zip'
train_metadata_path = f'{datasets_directory}/yale_faces/train_metadata.json'
test_zip_file_path = f'{datasets_directory}/yale_faces/test.zip'
test_metadata_path =  f'{datasets_directory}/yale_faces/test_metadata.json'
working_directory = './content/yale_faces'
data_extract_path = f'{working_directory}/datasets'
train_data_path = f'{data_extract_path}/train'
test_data_path = f'{data_extract_path}/test'
models_path = f'{working_directory}/models'
model_yml_path = models_path + '/lbph_classifier.yml'

## Load and extract training and test sets from zip files

In [ ]:
create_empty_directory(train_data_path)
train_zipe_file = zipfile.ZipFile(file=train_zip_file_path, mode = 'r')
train_zipe_file.extractall(train_data_path)
train_zipe_file.close()

./content/yale_faces/datasets/train directory was created


In [ ]:
create_empty_directory(test_data_path)
test_zipe_file = zipfile.ZipFile(file=test_zip_file_path, mode = 'r')
test_zipe_file.extractall(test_data_path)
test_zipe_file.close()

./content/yale_faces/datasets/test directory was created


## Load metadata files

In [ ]:
train_metadata = None
with open(train_metadata_path) as json_file:
    train_metadata = json.load(json_file)

In [ ]:
test_metadata = None
with open(test_metadata_path) as json_file:
    test_metadata = json.load(json_file)

## Defining helper classes

In [ ]:
class FaceImages:
  def __init__(self):
    self.images = []

  def get_npimages(self):
    images = []
    for image in self.images:
      np_image = image.image
      images.append(np_image)
    return images

  def get_labels(self):
    labels = []
    for image in self.images:
      label = image.label
      labels.append(label)

    return np.array(labels)

class FaceImage:
  def __init__(self, label, image):
    self.label = label
    self.image = image
    self.prediction = None

def get_image_paths(extract_path):
  image_paths = os.listdir(extract_path)
  invalid_paths = []
  for path in image_paths:
    if not ("subject" in path):
      print(f'Invalid path "{path}"')
      invalid_paths.append(path)

  for path in invalid_paths:
    image_paths.remove(path)
    print(f'Removed path "{path}"')

  return image_paths

def get_image_metadata(image_name, metadata):
  for image_metadata in metadata['images']:
      if image_name == image_metadata['fileName']:
        return image_metadata

  raise Exception(f'No metadata was found for image {image_name}')

def get_image_data(extract_path, metadata):
  image_paths = get_image_paths(extract_path)
  paths = [os.path.join(extract_path, f) for f in image_paths]
  images = FaceImages()

  for path in paths:
    image = Image.open(path).convert('L')
    image_np = np.array(image, 'uint8')

    image_name = os.path.split(path)[1].replace('.jpeg', '')
    image_metadata = get_image_metadata(image_name, metadata)

    label = image_metadata['label']
    images.images.append(FaceImage(label, image_np))

  return images

class YaleDataset:

  def load(preview):
    train_set = get_image_data(train_data_path, train_metadata)
    test_set =  get_image_data(test_data_path, test_metadata)

    if preview == True:
      sample_image = train_set.images[0]
      print(f'Image label is "{sample_image.label}"')
      cv2_imshow(sample_image.image)

    return train_set, test_set

## Datasets load test

In [ ]:
#YaleDataset.load(preview=True)